In [1]:
import os
from osgeo import ogr, osr

from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet

client = MalardClient()


proj4 = client.getProjection(DataSet( "cryotempo", "AntarcticaV2b", "antarctic"  )).proj4
print(proj4)


daShapefile = "/data/eagle/team/shared/Work/CryoTEMPO/MaskShape/CryoSat_Mask_4.0-polygon.shp"
#daShapefile = "/data/puma1/scratch/mtngla/chucach/Chucach-polygon.shp"

#kmz file
#daShapeFile = "/data/mouse1/team/jon/Anarctica/Coastline_Antarctica_v02.shp"


# Save extent to a new Shapefile
outShapefile = "SARIN_Antarctica.shp"

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(daShapefile) # 0 means read-only. 1 means writeable.
feature = None

layer = dataSource.GetLayer(0)

for i in range(0, layer.GetFeatureCount()):
    f = layer.GetFeature(i)
    
    if str(f.GetField(1)).startswith("LRM"):
        print( f.GetField(1))
    
    if f.GetField(1) == "SARIN over Greenland Coast":    #SARIN over Greenland Coast, LRM over Antarctica ice sheet, LRM over Greenland ice sheet,  
        feature = f
        print(f.GetField(1))
    if f.GetField(1) == "SARIN over Greenland Coast":
        print(f.GetGeometryRef().GetEnvelope())


#greenlandExtent = (-81.265, -6.8349, 59.1676, 84.2163)

        



+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs
LRM over Antarctica ice sheet
LRM over Greenland ice sheet


In [ ]:


sourceprj = layer.GetSpatialRef()

srs = osr.SpatialReference()
srs.ImportFromProj4(proj4)

targetprj = srs
transform = osr.CoordinateTransformation(sourceprj, targetprj)
        
        
outDriver = ogr.GetDriverByName("ESRI Shapefile")

# Remove output shapefile if it already exists
if os.path.exists(outShapefile):
    outDriver.DeleteDataSource(outShapefile)

# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
outlayer = outDataSource.CreateLayer("lrm_greenland", targetprj,geom_type=ogr.wkbPolygon)

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
outlayer.CreateField(idField)

# Create the feature and set values
transformed = feature.GetGeometryRef()
transformed.Transform(transform)

geom = ogr.CreateGeometryFromWkb(transformed.ExportToWkb())
defn = outlayer.GetLayerDefn()
feat = ogr.Feature(defn)
feat.SetGeometry(geom)
outlayer.CreateFeature(feat)

# Save and close DataSource
inDataSource = None
outDataSource = None